In [83]:
import numpy as np
import pandas as pd
import os
from collections import Counter

In [95]:
# Load cleaned text and metadata datasets
text_df = pd.read_csv('csv/segmented_cleaned.csv')
meta_df = pd.read_excel('csv/032818_RAC_Networks_Database.xlsx')

# Extract Volume and Letter Numbers

In [88]:
vol_lookup = {}

# Create lookup table mapping filename to volume number
for volume_dir in os.listdir('First Law Scans Cropped/'):
    # Make sure we don't iterate over .DS_Store
    if volume_dir != '.DS_Store':
        if 'III' in volume_dir:
            for filename in os.listdir('First Law Scans Cropped/' + volume_dir):
                if filename != '.DS_Store':
                    vol_lookup[filename] = 3
        elif 'II' in volume_dir:
            for filename in os.listdir('First Law Scans Cropped/' + volume_dir):
                if filename != '.DS_Store':
                    vol_lookup[filename] = 2
        elif 'I' in volume_dir:
            for filename in os.listdir('First Law Scans Cropped/' + volume_dir):
                if filename != '.DS_Store':
                    vol_lookup[filename] = 1

In [93]:
def extract_vol(filename):
    '''Extract Vol for each text'''
    return vol_lookup[filename]

def extract_letter_number(section_title):
    '''Extract letter number from section title'''
    return section_title.split('.')[0]

In [94]:
assert(sum(text_df['filename'].apply(extract_vol).isnull() == 0))

In [96]:
# Add columns for Vol and LetterNo 
text_df['Vol'] = text_df['filename'].apply(extract_vol)
text_df['LetterNo'] = text_df['section_title'].apply(extract_letter_number)

In [97]:
text_df.to_csv('csv/segmented_with_numbers.csv',index=False)

# Join with Metadata dataframe on Volume and Letter Number

In [98]:
# Read cleaned dataframes
text_df = pd.read_csv('csv/segmented_with_numbers.csv')
meta_df = pd.read_excel('csv/032818_RAC_Networks_Database.xlsx')

In [99]:
# Join dataframes on Letter Number and Volume
merged_df = pd.merge(meta_df, text_df, on=['Vol', 'LetterNo'])

In [100]:
# Filter out for a subset of the columns
merged_filtered = merged_df[[u'UID', u'Vol', u'LetterNo', u'Sender',  u'Place Sent From', u'Ship Name', u'Place Going To', u'Date',
          u'Boat/Fort', u'RAC/Other Nation', u'text', u'section_title', u'filename']].copy()

In [101]:
def remove_non_ascii(text):
    '''Remove non-ASCII characters (mostly due to OCR parsing error)'''
    if text is not np.nan:
        return ''.join(i for i in text if ord(i)<128)
    else:
        return np.nan
    
# Strip away non-ASCII characters from columns
for col in merged_filtered.columns:
    if merged_filtered[col].dtype == np.object and col != 'Date':
        merged_filtered[col] = merged_filtered[col].apply(remove_non_ascii)

In [102]:
# Export joined dataframe to csv file
merged_filtered.to_csv('csv/metadata_text_merged.csv', index=False)

# Identify UIDs with duplicates for manual cleanup

In [104]:
UID_counter = Counter(merged_filtered['UID'])

In [105]:
for uid, count in UID_counter.items():
    if count > 1:
        print uid

2
6
7
13
36
45
193
244
347
574
575
602
603
604
605
606
607
608
1641
1647
1655
1688
2860
2861
3019
3020


In [110]:
# Load cleaned merged file with manually deduped UIDs
merged_deduped = pd.read_csv('csv/metadata_text_merged_uid_dedup.csv')

UID_counter = Counter(merged_deduped['UID'])

# Check to see there's no duplicates
for uid, count in UID_counter.items():
    if count > 1:
        print uid